# 1 Time Value of Money

In [35]:
import plotly.graph_objects as go
import math
import sympy as sp
import scipy.optimize as sco
import numpy as np
import pandas as pd

## (1a) plot cf diagram

In [36]:

cf=[-100000]+[0]*3+[10000,10000]+[20000,20000]+[50000,50000]+[60000]  
df=[1.1**i for i in range(11)]

figure=go.Figure()

figure.add_trace(go.Bar(x=list(range(11)),y=cf,marker_color=['red']+['green']*10,text=cf,textposition='auto'))
figure.update_layout(xaxis_title='Years',yaxis_title='cf',title_text='cf diagram')

## (1b) Discrete vs Continuous

In [37]:
#discrete discounting
npv=0
for (i,j) in zip(cf,df):
    npv+=i/j
    
npv

#continuous discounting
npv_con=0
for (z,i,j) in zip(cf,range(11),[0.1]*11):
    npv_con+=z*math.e**(-i*j)

npv_con,npv

(-1455.8564816309881, 2254.8360448146923)

In the countinuos case we shouldn't invest in the project (npv<0), whereas in the discrete case we should invest in the project (npv>0). 

Discounting effect is more pronounced as the frequency of discounting (or compounding) increases, assuming discount rates are same in both cases. Since continuous discounting is a limit case of discrete discounting with infinitely frequent reinvesting of infinitely small payments, discounting (compounding) effect is maximal in the continuous model.

## (1c) Annual rates of return

In [38]:
# pairs of discount factors for discrete and continuous cases 
[(round((1.1**i)-1, 3), round(math.e**(i*0.1)-1,3)) for i in range(1,11)]

[(0.1, 0.105),
 (0.21, 0.221),
 (0.331, 0.35),
 (0.464, 0.492),
 (0.611, 0.649),
 (0.772, 0.822),
 (0.949, 1.014),
 (1.144, 1.226),
 (1.358, 1.46),
 (1.594, 1.718)]

In [39]:
# Differences between discrete and continuous discount factors
[(1.1**i) - math.e**(i*0.1)  for i in range(1,11)]

[-0.005170918075647624,
 -0.011402758160169668,
 -0.01885880757600278,
 -0.027724697641269946,
 -0.03821127070012764,
 -0.05055780039050828,
 -0.06503560747047543,
 -0.08195211849246586,
 -0.1016554201569475,
 -0.12453936835904278]

As we can see, discount factors of continuous discounting are always bigger than those of discrete discounting. Difference between two types of discounts factors increase with time.

## (1d) 10y US Treasury bond

Yield of 3% means that the market values the cash flow from the 10y note as if the average (geometric mean) interest rate in the next 10 years will be about 3%. Assuming interest rates of 10% over the next 10 years means that we are overestimating risk-free interest rate. Keeping the discount rate at 10% we will constantly downgrade the NPV of the project, and it may appear as a bed investment even if it is not so. Our suggestion is to reduce discount rates to 3% in order to obtain a picture of expected risk-free rate based on the market view.


# 2 Bond Pricing, part I

## (2a) Price, current yield and yield to maturity

In [40]:
rates=[2/100,2.5/100,3/100,3.5/100,4/100]
cf=[100*.04]*(len(rates)-1) # list of coupon payments with $4% coupon rate with a face value of $100
cf.append(104) # principal + the last coupon
cf

[4.0, 4.0, 4.0, 4.0, 104]

In [41]:
discounted_flow=[]

for i in range(len(rates)):
    discounted_flow.append(cf[i]/(1+rates[i])**(i+1))
print("bond price (discrete discounting): $"+str(sum(df)))

bond price (discrete discounting): $18.531167061100007


In [42]:
# current yield = coupon / market values   in percents

(100*.04)/sum(discounted_flow) * 100

3.9858271613659255

In [43]:
# yield to maturity

y=sp.Symbol('y',real='True',positive=True)
sp.solve(sp.Eq(lhs=sum([cf[i]/(1+y)**(i+1) for i in range(len(cf))]),rhs=sum(discounted_flow)),y)[0]*100 

#yield to maturity 3.92%

3.92030563261448

## (2b) Premium or discount

This is a premium bond since bond's value is above its par value.

## (2c) Price and yield, semi-annually payments

We have to make an assumption regarding expected returns at times which are non-integer numbers of years, i.e. at 0.5 years, 1.5 years etc. We will use linear interpolation (rates increase linearly for 25 basis points every 6 months). We will assume IR at time 0 is 0, hence rate for 6m maturity is 1%.

In [44]:

rates_6=[1/100,2/100,2.25/100,2.5/100,2.75/100,3/100,3.25/100,3.5/100,3.75/100,4/100]# interpolated
cf_6=[100*.04/2]*(len(rates_6)-1) # coupons
cf_6.append(102) # principal
periods=[0.5*i for i in range(1,len(rates_6)+1)] 

price=0
for i in range(len(cf_6)):
  price+=cf_6[i]/((1+rates_6[i])**periods[i])
print(price)

100.55028247878369


In [45]:
#this is still a premium bond
#current yield is slightly lower than yield to maturity
print("current yield for semi-annual payments: "+str(4/price))

current yield for semi-annual payments: 0.03978109162293013


In [46]:
#ytm calculation

y=sp.Symbol('y')
#define the NPV for ytm which we solve for x(ytm)
def npv(ytm):
  return sum(np.array(cf_6)*np.array([1/(1+ytm)**i for i in np.arange(0.5,5.5,0.5)]))


In [47]:
npv(y)

102.0*(y + 1)**(-5.0) + 2.0*(y + 1)**(-4.5) + 2.0*(y + 1)**(-4.0) + 2.0*(y + 1)**(-3.5) + 2.0*(y + 1)**(-3.0) + 2.0*(y + 1)**(-2.5) + 2.0*(y + 1)**(-2.0) + 2.0*(y + 1)**(-1.5) + 2.0*(y + 1)**(-1.0) + 2.0*(y + 1)**(-0.5)

In [48]:

lista_za_ytm=[]
#we use try and error approach.

"The second element of every sublist of list named lista_ya_ytm should be as close as possible to zero."
"The first element of every sublist is ytm. We are choosing the ytm for which the second element is the closest to zero" 
for i in np.arange(0.02,0.04,0.0001):
  lista_za_ytm.append([i,npv(i)-price])

minimal = min([n[1] for n in lista_za_ytm  if n[1]>0])


In [49]:
[n[1] for n in lista_za_ytm].index(minimal)#positon of the rate which makes the second element of every sublist the closest to zero

lista_za_ytm[191][1]*100 #ytm is lower

2.4230531825793378

## (2d)

In [50]:
#d
rates_1=[i+.5/100 for i in rates]

df_1=[]

cf=[100*.04]*(len(rates)-1)
cf.append(104)

cf

for i in range(1, len(rates_1)+1):
    df_1.append(cf[i-1]/(1+rates_1[i-1])**i)
print(sum(df_1))# new price in discrete case



(100*.04)/sum(df_1)# new current yield


y=sp.Symbol('y',real='True')
sol_1=sp.solve(sp.Eq(lhs=sum(np.array(cf)*np.array([1/(1+y)**i for i in range(1,6)])),rhs=sum(df_1)),y)#ytm in discrete case

sol_1

#this is a discount bond since the ytm is bigger than coupon rate
#bond price is below its pair value so this is a discount bond

98.15471908378738


[0.0441939586735344]

# 3 bond Pricing, part II

In [51]:
cf_10years_10=[10 for i in range(9)]

cf_10years_10.append(110)
mr=20/100
df_cf_20_10=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_20_10.append(cf_10years_10[i-1]/((1+mr)**i))


mr_19=19/100
df_cf_19_10=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_19_10.append(cf_10years_10[i-1]/((1+mr_19  )**i))


mr_21=21/100
df_cf_21_10=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_21_10.append(cf_10years_10[i-1]/((1+mr_21)**i))


cf_10years_20=[20 for i in range(9)]

cf_10years_20.append(120)
mr=20/100
df_cf_20_rate_20=[]
for i in range(1,len(cf_10years_20)+1):
  df_cf_20_rate_20.append(cf_10years_20[i-1]/((1+mr)**i))


mr_19=19/100
df_cf_19_rate_20=[]
for i in range(1,len(cf_10years_20)+1):
  df_cf_19_rate_20.append(cf_10years_20[i-1]/((1+mr_19)**i))


mr_21=21/100
df_cf_21_rate_20=[]
for i in range(1,len(cf_10years_20)+1):
  df_cf_21_rate_20.append(cf_10years_20[i-1]/((1+mr_21)**i))

cf_10years_30=[30 for i in range(9)]

cf_10years_30.append(130)
mr=20/100
df_cf_20_rate_30=[]
for i in range(1,len(cf_10years_30)+1):
  df_cf_20_rate_30.append(cf_10years_30[i-1]/((1+mr)**i))

mr_19=19/100
df_cf_19_rate_30=[]
for i in range(1,len(cf_10years_30)+1):
  df_cf_19_rate_30.append(cf_10years_30[i-1]/((1+mr_19)**i))

mr_21=21/100
df_cf_21_rate_30=[]
for i in range(1,len(cf_10years_30)+1):
  df_cf_21_rate_30.append(cf_10years_30[i-1]/((1+mr_21)**i))


data={'Bond':['A','B','C'],'Coupon rate':[10,20,30],'Maturity':[10]*3,'Price at 20%':[sum(df_cf_20_10),sum(df_cf_20_rate_20),sum(df_cf_20_rate_30)],'Price at 19%':[sum(df_cf_19_10),sum(df_cf_19_rate_20),sum(df_cf_19_rate_30)],'Change % down':[(sum(df_cf_19_10)-sum(df_cf_20_10))/sum(df_cf_20_10),(sum(df_cf_19_rate_20)-sum(df_cf_20_rate_20))/sum(df_cf_20_rate_20),(sum(df_cf_19_rate_30)-sum(df_cf_20_rate_30))/sum(df_cf_20_rate_30)],'Price at 21%':[sum(df_cf_21_10),sum(df_cf_21_rate_20),sum(df_cf_21_rate_30)],'Change % up':[(sum(df_cf_21_10)-sum(df_cf_20_10))/sum(df_cf_20_10),(sum(df_cf_21_rate_20)-sum(df_cf_20_rate_20))/sum(df_cf_20_rate_20),(sum(df_cf_21_rate_30)-sum(df_cf_20_rate_30))/sum(df_cf_20_rate_30)]}

pd.DataFrame(data)

"We can see that every number in change up is bigger in absolute value in comparisson to change down"


'We can see that every number in change up is bigger in absolute value in comparisson to change down'

In [52]:

"We can see that every number in change up is bigger in absolute value in comparisson to change down. Moreover the bigger  coupon the lower pct change of bond price in absolute is "

pd.DataFrame(data)

,Bond,Coupon rate,Maturity,Price at 20%,Price at 19%,Change % down,Price at 21%,Change % up
0,A,10,10,58.075279,60.949586,0.049493,55.405142,-0.045977
1,B,20,10,100.000000,104.338935,0.043389,95.945922,-0.040541
2,C,30,10,141.924721,147.728284,0.040892,136.486702,-0.038316


In [53]:

cf_10years_10=[10 for i in range(9)]

cf_10years_10.append(110)
mr=20/100
df_cf_20=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_20.append(cf_10years_10[i-1]/((1+mr)**i))

cf_10years_10=[10 for i in range(9)]

cf_10years_10.append(110)
mr_15=15/100
df_cf_15=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_15.append(cf_10years_10[i-1]/((1+mr_15)**i))

cf_10years_10=[10 for i in range(9)]

cf_10years_10.append(110)
mr_10=10/100
df_cf_10=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_10.append(cf_10years_10[i-1]/((1+mr_10)**i))

cf_10years_10=[10 for i in range(9)]

cf_10years_10.append(110)
mr_5=5/100
df_cf_5=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_5.append(cf_10years_10[i-1]/((1+mr_5)**i))

cf_10years_10=[10 for i in range(9)]

cf_10years_10.append(110)
mr_25=25/100
df_cf_25=[]
for i in range(1,len(cf_10years_10)+1):
  df_cf_25.append(cf_10years_10[i-1]/((1+mr_25)**i))

print(sum(df_cf_25),sum(df_cf_20),sum(df_cf_15),sum(df_cf_10),sum(df_cf_5))

fig=go.Figure()
fig.add_traces(go.Scatter(x=list(range(25,5,-5)),y=[sum(df_cf_25),sum(df_cf_20),sum(df_cf_15),sum(df_cf_10),sum(df_cf_5)]))

"On the graph we can see that the relationship between bond prices and discount rates is concave."

46.442450944 58.07527914449231 74.90615687072889 99.99999999999994 138.608674645924


'On the graph we can see that the relationship between bond prices and discount rates is concave.'

In [54]:
3#b


cf=[]
lista=[9,19,29]
cf_10=[]
cf_20=[]

cf_30=[]

for i in range(len(lista)):
  cf.append([10]*lista[i])
  
cf_10=cf[0]

cf_20=cf[1]

cf_30=cf[2]

cf_10.append(110)

cf_20.append(110)

cf_30.append(110)



price_10=0
price_20=0
price_30=0

for i in range(len(cf_10)):
  price_10+=cf_10[i]/(1.2)**(i+1)
for i in range(len(cf_20)):
  price_20+=cf_20[i]/(1.2)**(i+1)
for i in range(len(cf_30)):
  price_30+=cf_30[i]/(1.2)**(i+1)

price_10_19=0
price_20_19=0
price_30_19=0

for i in range(len(cf_10)):
  price_10_19+=cf_10[i]/(1.19)**(i+1)
for i in range(len(cf_20)):
  price_20_19+=cf_20[i]/(1.19)**(i+1)
for i in range(len(cf_30)):
  price_30_19+=cf_30[i]/(1.19)**(i+1)


price_10_21=0
price_20_21=0
price_30_21=0


for i in range(len(cf_10)):
  price_10_21+=cf_10[i]/(1.21)**(i+1)
for i in range(len(cf_20)):
  price_20_21+=cf_20[i]/(1.21)**(i+1)
for i in range(len(cf_30)):
  price_30_21+=cf_30[i]/(1.21)**(i+1)


import pandas as pd

data1={'Maturity':[10,30],'Price at 20':[price_10,price_30],'Price at 19':[price_10_19,price_30_19],'Price at 21':[price_10_21,price_30_21],'Pct_change_20_to_19':[(price_10_19-price_10)/price_10,(price_30_19-price_30)/price_30],'Pct_change_20_to_21':[(price_10_21-price_10)/price_10,(price_30_21-price_30)/price_30]}


pd.DataFrame(data1)

"Absolute value of every pct_change in second row(labeled with 1) is bigger then every absolute value of pct_change in the first row(labeled with 0)"
"This proves the higher sensitivity with respect to discount rate change of the bond with longer tensors in comparisson to the bond with rshorter tensors "

pd.DataFrame(data1)

,Maturity,Price at 20,Price at 19,Price at 21,Pct_change_20_to_19,Pct_change_20_to_21
0,10,58.075279,60.949586,55.405142,0.049493,-0.045977
1,30,50.210636,52.888075,47.791081,0.053324,-0.048188


# 4 Accrued Interest

In [55]:
par=100
coupon_rate=0.04/2
y=0.03/2

from datetime import date

d0 = date(2022, 5, 15)
d1 = date(2022, 7, 5)
d2 = date(2024, 11, 15)
delta = d1-d0 # days until next payment
delta_1 = d2-d0 # days until maturity
print(delta.days, delta_1.days)


51 915


## (4a) Compute the accrued interest

In [56]:
AIR = delta.days/(365/2) # accured interest
AIR

0.27945205479452057

## (4b) Calculate the clean and dirty price

In [57]:
cf_3=[par*coupon_rate]*5
cf_3.append(102)

rates_3=[(1+y)**(i/10) for i in list(range(5,35,5))]

df_3=[]

for i in range(len(rates_3)):
    df_3.append(cf_3[i]/rates_3[i])

In [58]:
print(delta.days/delta_1.days*2) #accured interest
print(sum(df_3))#clean bond price
((1+y)**(delta.days/delta_1.days))+sum(df_3)#dirty bond price

0.11147540983606558
107.32402145533423


108.32485165685071

In [59]:
#5


periods=list(range(1,11))#periods
rate=[0.02,0.025,0.03,0.035,0.04,0.0425,0.045,0.047,0.048,0.0485]# spot rates

fr=[]#forward rates
for i in range(1,len(periods)):
    a=((rate[i]*periods[i])-(rate[i-1]*periods[i-1]))/(periods[i]-periods[i-1])
    fr.append(a)

#a

print(fr)#forward rates

 #c   
(((3/100)*3)-((2.25/100)*1.5))/1.5 #forward rate 1.5 3 under the asumption that interest rates increase linearly


#b graphical display
import plotly.graph_objects as go


fig=go.Figure()

fig.add_trace(go.Scatter(y=fr,name="Forward rates"))

fig.add_trace(go.Scatter(y=rate,name="Spot rates"))


#forward rates are above spot rates in every period


"When the spot rates are upward sloping the forward rates are above spot rates. This is anology with average and marginal values. In order for average to rise marginal rate must be above average"

fig=go.Figure()

fig.add_trace(go.Scatter(y=fr,x=list(range(1,10)),name="Forward rates"))

fig.add_trace(go.Scatter(y=rate,name="Spot rates"))


[0.030000000000000002, 0.039999999999999994, 0.05000000000000002, 0.06, 0.05499999999999999, 0.06, 0.061, 0.055999999999999994, 0.05299999999999999]


In [60]:
rate[9]

0.0485